# Fraud Detection

## Data Analysis

In [ ]:
import glob
import pandas as pd

input_filenames = sorted(glob.glob('fraud_detection/*.Inputs'))
target_filenames = sorted(glob.glob('fraud_detection/*.Targets'))

df = pd.DataFrame()
for input_file, target_file in zip(input_filenames, target_filenames):
    df = df.append(pd.read_csv(input_file))
    df['fraud'] = pd.read_csv(target_file)

df.head()

### Dataset Profiling Report

In [ ]:
import pandas_profiling

profile = pandas_profiling.ProfileReport(df)
profile.to_file(output_file="ProfilingResult.html")

In [ ]:
profile

### Rows containing null values

In [ ]:
df[pd.isnull(df).any(axis=1)]

## Data Preprocessing

Drop highly correlated columns

In [ ]:
clean_df = df.drop(columns=['hour2', 'domain1', 'total'])
clean_df.head()

In [ ]:
clean_df = clean_df.dropna()
clean_df = clean_df.drop_duplicates()

In [ ]:
clean_df['state1'] = clean_df['state1'].astype('category')
clean_df['state1'] = clean_df['state1'].cat.codes

clean_df.head()

In [ ]:
clean_df.shape

## Split dataset

In [ ]:
from sklearn.model_selection import train_test_split

X = clean_df.drop(columns=['fraud'])
y = clean_df['fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [ ]:
import pandas_profiling

profile = pandas_profiling.ProfileReport(df_train)
profile.to_file(output_file="PreprocessedProfilingResult.html")

## Oversampling & Undersampling

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X_res, y_res = smote.fit_resample(X_train, y_train)

dataset_res = []
for i in range(len(y_res)):
    dataset_res.append(np.append(X_res[i], y_res[i]))

dataset_res = np.asarray(dataset_res)
dataset_res.shape

In [ ]:
columns = df_train.columns

oversampled_df = pd.DataFrame(dataset_res, columns=columns)
oversampled_df

In [ ]:
import pandas_profiling

profile = pandas_profiling.ProfileReport(oversampled_df)
profile.to_file(output_file="OversampledProfilingResult.html")

In [ ]:
oversampled_df['fraud'].value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

X = oversampled_df.drop(columns=['fraud']).to_numpy()
y = oversampled_df['fraud']

model = MultinomialNB(alpha=1e-10)

kf = StratifiedKFold(n_splits=10)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print (y_train)
    print (X_train)
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    print ("Accuracy\t", accuracy_score(y_test, predict))

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

X = oversampled_df.drop(columns=['fraud']).to_numpy()
y = oversampled_df['fraud']

model = Sequential()
model.add(Dense(50, input_dim=16, init='uniform', activation='relu'))
model.add(Dense(25, init='uniform', activation='relu'))
model.add(Dense(10, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

kf = StratifiedKFold(n_splits=2)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    history = model.fit(X_train, y_train, nb_epoch=50, batch_size=50,  verbose=1)
    predict = model.predict_classes(X_test)
    print ("Accuracy\t", accuracy_score(y_test, predict))

In [ ]:
dataset_res[0]

In [ ]:
X_train.columns